In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# counties_info_df = pd.read_csv("../Data/counties.csv")
# counties_pop_df = counties_info_df[["fips","population"]]
# counties_pop_df.set_index("fips",inplace=True)
# counties_pop_df.head()

In [3]:
def check_inconsistencies(df):
    fips_value = df['fips'].tolist()[0]
    global error_message
    
    subset_df = df[['actuals.newCases','actuals.cases']]
    previous_cumulative_cases = 0
    previous_new_cases = 0
    for index, row in subset_df.iterrows():
        if previous_cumulative_cases + previous_new_cases > row['actuals.cases']:
            message = f"Issue at time point: {index} for fips={fips_value}"
            error_message.append(message)
        
        previous_cumulative_cases = row['actuals.cases']
        previous_new_cases = row['actuals.newCases']


In [12]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
global count_cleaned_df
global error_message
error_message= []
from tqdm import tqdm

def fill_missing_days_with_NA(df):
    df.index = pd.DatetimeIndex(df['date'].values)
    new_date_range = pd.date_range(start="2020-04-05", end="2022-05-07", freq="D")
    final_df = df.reindex(new_date_range)
    final_df['date'] = final_df.index

    return final_df


def data_preprocessing(df): 
    # columns_to_exclude = ['date', 'riskLevels.overall', 'riskLevels.caseDensity', 'cdcTransmissionLevel']
    columns_to_exclude = ['date']
    new_df = df.loc[:, ~df.columns.isin(columns_to_exclude)]
    
    #Filling missing values using backfill and forwardfill
    new_df.fillna(method="backfill", axis=None, inplace=True)
    new_df.fillna(method="ffill", axis=None, inplace=True)
    
    #Normalization using MinMaxScaler
    cols_to_norm = ['riskLevels.overall', 'riskLevels.caseDensity', 'cdcTransmissionLevel']
    scaler = MinMaxScaler()
    new_df[cols_to_norm] = scaler.fit_transform(new_df[cols_to_norm])
    
    #Copying Date
    new_df['date'] = df.date

    return new_df

def population_feature_engineering(df,fip):
    df["metrics.deathDensity"] = df['actuals.deaths']/counties_pop_df.loc[fip]["population"]
    return df


In [13]:
original_csv = pd.read_csv("./counties.finalfeatures.timeseries.csv")
fips_list = list(set(original_csv['fips'].tolist()))
county_df_list = []
# outlier_counties=[]

i=0
for fip in tqdm(fips_list):
    county_df = original_csv[original_csv['fips'] == fip]

    #STEP 1
    county_df = fill_missing_days_with_NA(county_df) 
    #dataframe is set to DateTimeIndex
    
    #STEP 2
    new_county_df = data_preprocessing(county_df)

    #STEP 3
#     new_feature_engg_df = population_feature_engineering(new_county_df, fip)
    
    #STEP 4
    county_df_list.append(new_county_df) #Appending the final dataframe for each county

print(len(county_df_list))

100%|██████████████████████████████████████████████████████████████████████████████| 3223/3223 [03:41<00:00, 14.52it/s]

3223


In [14]:
#Checking DataFrame before QC
baker = original_csv[original_csv['fips'] == 41001]
baker.head(15).tail(5)

,date,fips,state,county,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,metrics.testPositivityRatio
1750154,2020-03-31,41001,OR,Baker County,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,0.0
1750155,2020-04-01,41001,OR,Baker County,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,0.0
1750156,2020-04-02,41001,OR,Baker County,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,0.0
1750157,2020-04-03,41001,OR,Baker County,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,0.0
1750158,2020-04-04,41001,OR,Baker County,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN


In [16]:
#Checking DataFrame after QC
county_df_list[0].head(10).tail(10)

,fips,state,county,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,metrics.testPositivityRatio,date
2020-04-05,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-05
2020-04-06,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-06
2020-04-07,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-07
2020-04-08,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-08
2020-04-09,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-09
2020-04-10,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-10
2020-04-11,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-11
2020-04-12,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-12
2020-04-13,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-13
2020-04-14,41001,OR,Baker County,1.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.056,2020-04-14


## Concatenating list of county dataframes and Removing Outlier

In [18]:
import joblib
joblib.dump(county_df_list, "daily_agg_counties_timeseries_list.pkl")
# county_df_list = joblib.load("daily_agg_counties_timeseries_list.pkl")

['daily_agg_counties_timeseries_list.pkl']

In [19]:
## Sanity Check
import pandas as pd
total_rows = 0
for county in county_df_list:
    total_rows += county.shape[0]
print(total_rows)

combinedf = pd.concat(county_df_list)
print(combinedf.shape)

2459149
(2459149, 14)


In [22]:
combinedf
# combinedf.to_csv("daily_agg_counties_timeseries_combined.csv")

,fips,state,county,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,metrics.testPositivityRatio,date
2020-04-05,41001.0,OR,Baker County,1.0,0.0,0.0,0.007,0.030,1.00,0.0,0.0,0.0,0.056,2020-04-05
2020-04-06,41001.0,OR,Baker County,1.0,0.0,0.0,0.007,0.030,1.00,0.0,0.0,0.0,0.056,2020-04-06
2020-04-07,41001.0,OR,Baker County,1.0,0.0,0.0,0.007,0.030,1.00,0.0,0.0,0.0,0.056,2020-04-07
2020-04-08,41001.0,OR,Baker County,1.0,0.0,0.0,0.007,0.030,1.00,0.0,0.0,0.0,0.056,2020-04-08
2020-04-09,41001.0,OR,Baker County,1.0,0.0,0.0,0.007,0.030,1.00,0.0,0.0,0.0,0.056,2020-04-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-03,24510.0,MD,Baltimore city,114248.0,1775.0,19.4,0.642,0.727,1.32,0.4,0.4,1.0,0.035,2022-05-03
2022-05-04,24510.0,MD,Baltimore city,114424.0,1775.0,21.5,0.643,0.728,1.33,0.4,0.4,1.0,0.038,2022-05-04
2022-05-05,24510.0,MD,Baltimore city,114683.0,1775.0,23.6,0.643,0.728,1.34,0.4,0.4,1.0,0.041,2022-05-05
2022-05-06,24510.0,MD,Baltimore city,114991.0,1776.0,27.2,0.643,0.728,1.35,0.6,0.6,1.0,0.044,2022-05-06


## Aggregation

## Features to Aggegrate by sum:
- actuals.cases
- actuals.deaths
- metrics.deathDensity
- metrics.caseDensity
- metrics.testPositivityRatio

## Features to avergae, not sum
- cdcTransmissionLevel
- riskLevels.caseDensity
- riskLevels.overall
- metrics.vaccinationsCompletedRatio
- metrics.vaccinationsInitiatedRatio

In [23]:
combinedf['date']=combinedf.index
agg_sum_df = combinedf.groupby(['date'])['actuals.cases', 'actuals.deaths', 'metrics.caseDensity'].sum()

agg_means_df = combinedf.groupby(['date'])['metrics.testPositivityRatio','cdcTransmissionLevel', 'riskLevels.caseDensity', 'riskLevels.overall',
                                          'metrics.vaccinationsCompletedRatio','metrics.vaccinationsInitiatedRatio'].mean()
agg_df = pd.merge(agg_sum_df,agg_means_df, left_index=True,right_index=True)

In [24]:
agg_sum_df

,actuals.cases,actuals.deaths,metrics.caseDensity
date,,,
2020-04-05,332871.0,9285.0,9703.1
2020-04-06,363389.0,10662.0,10447.0
2020-04-07,394848.0,12748.0,11193.2
2020-04-08,426315.0,14655.0,11619.4
2020-04-09,460667.0,16262.0,11993.1
...,...,...,...
2022-05-03,80929784.0,984403.0,42303.7
2022-05-04,81038153.0,986513.0,43943.2
2022-05-05,81113986.0,987075.0,45700.7


In [25]:
agg_means_df

,metrics.testPositivityRatio,cdcTransmissionLevel,riskLevels.caseDensity,riskLevels.overall,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
date,,,,,,
2020-04-05,0.108485,0.431430,0.110518,0.204871,0.052442,0.117057
2020-04-06,0.107930,0.433266,0.115126,0.209339,0.052442,0.117057
2020-04-07,0.108964,0.437093,0.122153,0.217018,0.052442,0.117057
2020-04-08,0.108964,0.445548,0.125287,0.219051,0.052442,0.117057
2020-04-09,0.108509,0.448211,0.127071,0.219904,0.052442,0.117057
...,...,...,...,...,...,...
2022-05-03,0.076613,0.574801,0.277412,0.325008,0.522983,0.588492
2022-05-04,0.079808,0.589151,0.280546,0.328995,0.523095,0.588607
2022-05-05,0.083186,0.603242,0.284455,0.335759,0.523285,0.588795


In [26]:
agg_df

,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.testPositivityRatio,cdcTransmissionLevel,riskLevels.caseDensity,riskLevels.overall,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
date,,,,,,,,,
2020-04-05,332871.0,9285.0,9703.1,0.108485,0.431430,0.110518,0.204871,0.052442,0.117057
2020-04-06,363389.0,10662.0,10447.0,0.107930,0.433266,0.115126,0.209339,0.052442,0.117057
2020-04-07,394848.0,12748.0,11193.2,0.108964,0.437093,0.122153,0.217018,0.052442,0.117057
2020-04-08,426315.0,14655.0,11619.4,0.108964,0.445548,0.125287,0.219051,0.052442,0.117057
2020-04-09,460667.0,16262.0,11993.1,0.108509,0.448211,0.127071,0.219904,0.052442,0.117057
...,...,...,...,...,...,...,...,...,...
2022-05-03,80929784.0,984403.0,42303.7,0.076613,0.574801,0.277412,0.325008,0.522983,0.588492
2022-05-04,81038153.0,986513.0,43943.2,0.079808,0.589151,0.280546,0.328995,0.523095,0.588607
2022-05-05,81113986.0,987075.0,45700.7,0.083186,0.603242,0.284455,0.335759,0.523285,0.588795


In [27]:
agg_df.to_csv("county_daily_aggregated_timeseries.csv")

In [28]:
agg_df_cases_deaths = agg_df[["actuals.deaths","metrics.caseDensity"]]
agg_df_cases_deaths

,actuals.deaths,metrics.caseDensity
date,,
2020-04-05,9285.0,9703.1
2020-04-06,10662.0,10447.0
2020-04-07,12748.0,11193.2
2020-04-08,14655.0,11619.4
2020-04-09,16262.0,11993.1
...,...,...
2022-05-03,984403.0,42303.7
2022-05-04,986513.0,43943.2
2022-05-05,987075.0,45700.7


In [29]:
agg_df_cases_deaths.to_csv("county_daily_aggregated_timeseries_cases_deaths.csv")